<a href="https://colab.research.google.com/github/Tushar-ml/Projects/blob/master/Copy_of_Colab_8_Deep_Reinforcement_Learning_for_Stock_Market_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Installing dependencies and environment setup


In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 53kB/s 
     |████████████████████████████████| 3.0MB 33.8MB/s 
     |████████████████████████████████| 419kB 45.5MB/s 


In [0]:
!pip install pandas-datareader

## Stage 2: Importing project dependencies

In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
tf.__version__

'2.2.0'

## Stage 3: Building the AI Trader network

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

## Stage 4: Dataset preprocessing

### Defining helper functions

#### Sigmoid

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

#### Price format function

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

#### Dataset loader

In [0]:
def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

### State creator

In [0]:
def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

### Loading a dataset

In [0]:
stock_name = "FB"
data = dataset_loader(stock_name)

## Stage 5: Training the AI Trader

### Setting hyper parameters

In [0]:
window_size = 10
episodes = 3

batch_size = 32
data_samples = len(data) - 1

### Defining the Trader model

In [0]:
trader = AI_Trader(window_size)

In [19]:
trader.model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_5 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_6 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Training loop

In [0]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      # a = stocks_price_format(data[t] - buy_price).split(' ')[-1]
      if (data[t]-buy_price)>0:
        print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      else:
        print("AI Trader sold: ", stocks_price_format(data[t]), " Loss: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/1256 [00:00<?, ?it/s]

Episode: 1/3
AI Trader bought:  $ 80.540001
AI Trader sold:  $ 79.330002  Loss: - $ 1.209999
AI Trader bought:  $ 80.290001
AI Trader bought:  $ 80.440002
AI Trader bought:  $ 82.440002
AI Trader bought:  $ 82.050003
AI Trader bought:  $ 82.139999
AI Trader bought:  $ 80.669998
AI Trader bought:  $ 80.669998
AI Trader bought:  $ 81.830002
AI Trader sold:  $ 81.529999  Profit: $ 1.239998
AI Trader bought:  $ 81.059998
AI Trader sold:  $ 81.790001  Profit: $ 1.349998
AI Trader sold:  $ 82.510002  Profit: $ 0.070000
AI Trader sold:  $ 84.739998  Profit: $ 2.689995
AI Trader sold:  $ 87.980003  Profit: $ 5.840004
AI Trader bought:  $ 88.010002
AI Trader bought:  $ 85.800003
AI Trader bought:  $ 85.769997
AI Trader bought:  $ 86.910004
AI Trader bought:  $ 87.290001
AI Trader sold:  $ 87.550003  Profit: $ 6.880005
AI Trader sold:  $ 87.220001  Profit: $ 6.550003
AI Trader sold:  $ 85.650002  Profit: $ 3.820000
AI Trader bought:  $ 85.879997


  3%|▎         | 33/1256 [00:03<02:02,  9.97it/s]

AI Trader sold:  $ 87.949997  Profit: $ 6.889999


  3%|▎         | 34/1256 [00:06<19:27,  1.05it/s]

...

 16%|█▌        | 198/1256 [07:58<50:50,  2.88s/it]

AI Trader sold:  $ 105.730003  Loss: - $ 3.849998


 16%|█▌        | 199/1256 [08:01<50:55,  2.89s/it]

AI Trader sold:  $ 105.930000  Loss: - $ 2.459999


 17%|█▋        | 210/1256 [08:33<50:03,  2.87s/it]

In [0]:

trader.model.save("ai_trader_{}_Apple.h5".format(episode))